# Análisis Exploratorio de Datos

### Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
font_path = font_manager.findfont("DejaVu Serif")
plt.rc('font', family='serif')
plt.rc('font', serif='DejaVu Serif')

import nltk
from nltk.util import ngrams

import numpy as np
np.set_printoptions(threshold=np.inf)

from scipy import stats
import pandas as pd
from collections import Counter
from wordcloud import WordCloud

### Carga de Datos limpios

In [ ]:
df_clean = pd.read_csv('../data/train_clean.csv')

### Cantidad de palabras

In [ ]:
# Calculamos cuantas palabras tiene cada noticia
df_clean['n_words'] = df_clean['text'].apply(lambda x: len(x.split()))

In [ ]:
# Calculamos algunas medidas de la cantidad de palabras por noticia
print('Reales \t -> Media:\t\t', 	df_clean[df_clean['label'] == 0]['n_words'].mean())
print('\t -> Varianza:\t\t', 		df_clean[df_clean['label'] == 0]['n_words'].var())
print('\t -> Desviación Típica:\t', df_clean[df_clean['label'] == 0]['n_words'].std())
print('')
print('Fake \t -> Media:\t\t', 		df_clean[df_clean['label'] == 1]['n_words'].mean())
print('\t -> Varianza:\t\t', 		df_clean[df_clean['label'] == 1]['n_words'].var())
print('\t -> Desviación Típica:\t', df_clean[df_clean['label'] == 1]['n_words'].std())
print('')
print('Total \t -> Media:\t\t', 	df_clean['n_words'].mean())
print('\t -> Varianza:\t\t', 		df_clean['n_words'].var())
print('\t -> Desviación Típica:\t', df_clean['n_words'].std())

#### Palabras / noticia

In [ ]:
# Hacemos un histograma de la cantidad de palabras por noticia
plt.figure(figsize=(8, 6))
bin_edges = np.arange(0, 3200, 200)
n, bins, patches = plt.hist(df_clean['n_words'], bins=bin_edges, color='#8294c4', rwidth=0.85)
plt.title('Cantidad de palabras por noticia')
plt.xlabel('Palabras')
plt.ylabel('Noticias')
plt.xlim(0, 3000)
plt.ylim(0, 7500)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_xticks(np.arange(0, 3000, 200))
# plt.xticks(rotation=30)
plt.gca().set_yticks(np.arange(0, 7500, 500))

# Calcular los porcentajes
porcentajes = [(100 * value / sum(n)) for value in n]

# Agregar etiquetas a las barras
for patch, porcentaje in zip(patches, porcentajes):
	height = patch.get_height()
	width = patch.get_width()
	if patch.get_x() + width / 2 + 10 < plt.xlim()[1]:
		plt.text(patch.get_x() + patch.get_width() / 2, height + 60, f'{porcentaje:.1f}%', ha='center', fontsize=7)
  
plt.savefig('../images/cantidad_palabras.png', bbox_inches='tight', dpi=300)
plt.show()

#### Palabras / Label

In [ ]:
# Hacemos un histograma de la cantidad de palabras por noticia en cada label
plt.figure(figsize=(8, 6))
bin_edges = np.arange(0, 3200, 50)
plt.hist(df_clean[df_clean['label'] == 0]['n_words'], bins=bin_edges, alpha=0.8, color='#6c78d4', label='Real', rwidth=0.85)
plt.hist(df_clean[df_clean['label'] == 1]['n_words'], bins=bin_edges, alpha=0.6, color='#faed57', label='Fake', rwidth=0.85)
plt.title('Cantidad de palabras en cada categoría')
plt.xlabel('Palabras')
plt.ylabel('Noticias')
plt.xlim(0, 2200)
plt.ylim(0, 2000)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_xticks(np.arange(0, 2200, 200))
plt.gca().set_yticks(np.arange(0, 2100, 200))
plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(100))
plt.legend()
plt.savefig('../images/cantidad_palabras_label.png', bbox_inches='tight', dpi=300)
plt.show()

#### Palabras distintas / noticia

In [ ]:
# Calculamos la cantidad de palabras distintas por cada noticia y las guarda en una nueva columna
df_clean['n_unique_words'] = df_clean['text'].apply(lambda x: len(set(x.split())))

In [ ]:
# Calculamos algunas medidas de la cantidad de palabras distintas
print('Reales \t -> Media:\t\t',		df_clean[df_clean['label'] == 0]['n_unique_words'].mean())
print('\t -> Varianza:\t\t',			df_clean[df_clean['label'] == 0]['n_unique_words'].var())
print('\t -> Desviación Típica:\t',	df_clean[df_clean['label'] == 0]['n_unique_words'].std())
print('')
print('Fake \t -> Media:\t\t',			df_clean[df_clean['label'] == 1]['n_unique_words'].mean())
print('\t -> Varianza:\t\t',			df_clean[df_clean['label'] == 1]['n_unique_words'].var())
print('\t -> Desviación Típica:\t',	df_clean[df_clean['label'] == 1]['n_unique_words'].std())
print('')
print('Total \t -> Media:\t\t',			df_clean['n_unique_words'].mean())
print('\t -> Varianza:\t\t',			df_clean['n_unique_words'].var())
print('\t -> Desviación Típica:\t',		df_clean['n_unique_words'].std())

In [ ]:
# Hacemos un histograma de la cantidad de palabras distintas por noticia
plt.figure(figsize=(8, 6))
bin_edges = np.arange(0, 3200, 100)
n, bins, patches = plt.hist(df_clean['n_unique_words'], bins=bin_edges, rwidth=0.85, color='#8294c4')
plt.title('Cantidad de palabras distintas por noticia')
plt.xlabel('Palabras distintas')
plt.ylabel('Noticias')
plt.xlim(0, 2000)
plt.ylim(0, 6000)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_xticks(np.arange(0, 2000, 200))
plt.gca().set_yticks(np.arange(0, 6000, 500))

# Calcular los porcentajes
porcentajes = [(100 * value / sum(n)) for value in n]

# Agregar etiquetas a las barras
for patch, porcentaje in zip(patches, porcentajes):
	height = patch.get_height()
	width = patch.get_width()
	if patch.get_x() + width / 2 + 10 < plt.xlim()[1]:
		plt.text(patch.get_x() + patch.get_width() / 2, height + 60, f'{porcentaje:.1f}%', ha='center', fontsize=7)
  
plt.savefig('../images/cantidad_palabras_distintas.png', bbox_inches='tight', dpi=300)
plt.show()

#### Palabras distintas / Label

In [ ]:
# Hacemos un histograma de la cantidad de palabras por noticia en cada label
plt.figure(figsize=(8, 6))
bin_edges = np.arange(0, 3200, 50)
plt.hist(df_clean[df_clean['label'] == 0]['n_unique_words'], bins=bin_edges, alpha=0.8, color='#6c78d4', label='Real', rwidth=0.85)
plt.hist(df_clean[df_clean['label'] == 1]['n_unique_words'], bins=bin_edges, alpha=0.6, color='#faed57', label='Fake', rwidth=0.85)
plt.title('Cantidad de palabras distintas en cada categoría')
plt.xlabel('Palabras distintas')
plt.ylabel('Noticias')
plt.xlim(0, 2200)
plt.ylim(0, 2000)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_xticks(np.arange(0, 2200, 200))
plt.gca().set_yticks(np.arange(0, 2100, 200))
plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(100))
plt.legend()
plt.savefig('../images/cantidad_palabras_distintas_label.png', bbox_inches='tight', dpi=300)
plt.show()

#### Palabras distintas / Palabras

In [ ]:
# Hacemos un gráfico de dispersión de la cantidad de palabras y cantidad de palabras distintas por noticia y categoría
plt.figure(figsize=(8, 6))
plt.scatter(df_clean[df_clean['label'] == 0]['n_words'], df_clean[df_clean['label'] == 0]['n_unique_words'], alpha=0.1, color='#6c78d4', label='Real')
plt.scatter(df_clean[df_clean['label'] == 1]['n_words'], df_clean[df_clean['label'] == 1]['n_unique_words'], alpha=0.1, color='#faed57', label='Fake')
plt.title('Cantidad de palabras y palabras distintas por noticia')
plt.xlabel('Palabras')
plt.ylabel('Palabras distintas')
plt.xlim(0, 3000)
plt.ylim(0, 2000)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_xticks(np.arange(0, 3000, 200))
plt.gca().set_yticks(np.arange(0, 2000, 200))
plt.gca().yaxis.set_minor_locator(plt.MultipleLocator(100))
plt.legend()

plt.savefig('../images/dispersion_palabras_distintas.png', bbox_inches='tight', dpi=300)
plt.show()

### Palabras más frecuentes en cada set

In [ ]:
# Utilizamos Counter para calcular la frecuencia de las palabras por label y en global
words = Counter()
words_real = Counter()
words_fake = Counter()
for idx, row in df_clean.iterrows():
	words.update(row['text'].split())
	if row['label'] == 0:
		words_real.update(row['text'].split())
	else:
		words_fake.update(row['text'].split())
	
# Pasamos a dataframes con palabras y cantidad
words_df = pd.DataFrame.from_dict(words, orient='index', columns=['count'])
words_real_df = pd.DataFrame.from_dict(words_real, orient='index', columns=['count'])
words_fake_df = pd.DataFrame.from_dict(words_fake, orient='index', columns=['count'])

# Utilizamos Counter para calcular la aparición de palabras por label y en global
words_appearance = Counter()
words_appearance_real = Counter()
words_appearance_fake = Counter()
for idx, row in df_clean.iterrows():
	words_appearance.update(set(row['text'].split()))
	if row['label'] == 0:
		words_appearance_real.update(set(row['text'].split()))
	else:
		words_appearance_fake.update(set(row['text'].split()))
  
# Añadimos estos datos a los dataframes
words_df['appearance'] = words_df.index.map(words_appearance)
words_real_df['appearance'] = words_real_df.index.map(words_appearance_real)
words_fake_df['appearance'] = words_fake_df.index.map(words_appearance_fake)

In [ ]:
# Los mostramos ordenados por cantidad
print('Palabras más frecuentes en noticias reales:')
print(words_real_df.sort_values(by='count', ascending=False).head(10))
print('Palabras más frecuentes en noticias falsas:')
print(words_fake_df.sort_values(by='count', ascending=False).head(10))
print('Palabras más frecuentes:')
print(words_df.sort_values(by='count', ascending=False).head(10))

In [ ]:
# Calculamos el porcentaje de noticias verdaderas que contienen cada palabra
words_real_df['percentage'] = words_real_df['appearance'] / len(df_clean[df_clean['label']==0]) * 100

In [ ]:
# Hacemos un gráfico de barras de las palabras más frecuentes en noticias reales
cantidad_palabras = 25
plt.figure(figsize=(8, 4))
plt.bar(words_real_df.sort_values(by='percentage', ascending=False).head(cantidad_palabras).index, words_real_df.sort_values(by='percentage', ascending=False).head(cantidad_palabras)['percentage'], color='#6c78d4')
plt.title('Palabras más aparecidas en noticias reales')
plt.ylim(0, 100)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_yticks(np.arange(0, 110, 10))
plt.xticks(rotation=90)

# Hacemos que en el eje y se muestre el símbolo de porcentaje
vals = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0%}'.format(x/100) for x in vals])

plt.savefig('../images/palabras_mas_frecuentes_reales_20.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Calculamos el porcentaje de noticias falsas que contienen cada palabra
words_fake_df['percentage'] = words_fake_df['appearance'] / len(df_clean[df_clean['label']==1]) * 100

In [ ]:
# Hacemos un gráfico de barras de las palabras más frecuentes en noticias falsas
cantidad_palabras = 25
plt.figure(figsize=(8, 4))
plt.bar(words_fake_df.sort_values(by='percentage', ascending=False).head(cantidad_palabras).index, words_fake_df.sort_values(by='percentage', ascending=False).head(cantidad_palabras)['percentage'], color='#faed57')
plt.title('Palabras más aparecidas en noticias falsas')
plt.ylim(0, 100)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_yticks(np.arange(0, 110, 10))
plt.xticks(rotation=90)

# Hacemos que en el eje y se muestre el símbolo de porcentaje
vals = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0%}'.format(x/100) for x in vals])

plt.savefig('../images/palabras_mas_frecuentes_falsas_20.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Creamos word_intersection para guardar las palabras que aparecen en ambos dataframes
words_intersection = words_real_df.merge(words_fake_df, how='inner', left_index=True, right_index=True, suffixes=('_real', '_fake'))

# Calculamos los ratios de aparición de las palabras en ambos dataframes
words_intersection['ratio_real'] = words_intersection['appearance_real'] / words_intersection['appearance_fake']
words_intersection['ratio_fake'] = words_intersection['appearance_fake'] / words_intersection['appearance_real']

In [ ]:
# Hacemos un gráfico de barras de las palabras con más ratio de aparición en noticias reales
cantidad_palabras = 25
plt.figure(figsize=(8, 4))
plt.bar(words_intersection.sort_values(by='ratio_real', ascending=False).head(cantidad_palabras).index, words_intersection.sort_values(by='ratio_real', ascending=False).head(cantidad_palabras)['ratio_real'], color='#6c78d4')
plt.title('Palabras con mayor ratio de aparición en noticias reales')
plt.ylabel('Ratio')
plt.ylim(0, 100)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_yticks(np.arange(0, 150, 10))
plt.xticks(rotation=90)

plt.savefig('../images/palabras_mas_ratio_reales_25.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Hacemos un gráfico de barras de las palabras con más ratio de aparición en noticias falsas
cantidad_palabras = 25
plt.figure(figsize=(8, 4))
plt.bar(words_intersection.sort_values(by='ratio_fake', ascending=False).head(cantidad_palabras).index, words_intersection.sort_values(by='ratio_fake', ascending=False).head(cantidad_palabras)['ratio_fake'], color='#faed57')
plt.title('Palabras con mayor ratio de aparición en noticias falsas')
plt.ylabel('Ratio')
plt.ylim(0, 140)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_yticks(np.arange(0, 150, 10))
plt.xticks(rotation=90)

plt.savefig('../images/palabras_mas_ratio_falsas_25.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Hacemos un wordcloud de las 20 palabras más aparecidas en noticias reales que están en words_real_df
wordcloud = WordCloud(width=1920, height=1080, background_color='white', max_words=200).generate_from_frequencies(words_real_df.to_dict()['appearance'])
plt.figure(figsize=(8,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')	
plt.title('Palabras más frecuentes en noticias reales')
plt.savefig('../images/palabras_mas_frecuentes_reales_wordcloud.png', bbox_inches='tight', dpi=300)
plt.show()

# Hacemos un wordcloud de las 20 palabras más aparecidas en noticias falsas que están en words_fake_df
wordcloud = WordCloud(width=1920, height=1080, background_color='white', max_words=200).generate_from_frequencies(words_fake_df.to_dict()['appearance'])
plt.figure(figsize=(8,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Palabras más frecuentes en noticias falsas')
plt.savefig('../images/palabras_mas_frecuentes_falsas_wordcloud.png', bbox_inches='tight', dpi=300)
plt.show()

### Bigramas

In [ ]:
# Utilizamos Counter para calcular la frecuencia de los bigramas por label y en global
bigrams = Counter()
bigrams_real = Counter()
bigrams_fake = Counter()
for idx, row in df_clean.iterrows():
	bigrams.update(ngrams(row['text'].split(), 2))
	if row['label'] == 0:
		bigrams_real.update(ngrams(row['text'].split(), 2))
	else:
		bigrams_fake.update(ngrams(row['text'].split(), 2))
	
# Pasamos a dataframes con ngramas y cantidad
bigrams_df = pd.DataFrame.from_dict(bigrams, orient='index', columns=['count'])
bigrams_real_df = pd.DataFrame.from_dict(bigrams_real, orient='index', columns=['count'])
bigrams_fake_df = pd.DataFrame.from_dict(bigrams_fake, orient='index', columns=['count'])

# Utilizamos Counter para calcular la aparición de bigramas por label y en global
bigrams_appearance = Counter()
bigrams_appearance_real = Counter()
bigrams_appearance_fake = Counter()
for idx, row in df_clean.iterrows():
	bigrams_appearance.update(set(ngrams(row['text'].split(), 2)))
	if row['label'] == 0:
		bigrams_appearance_real.update(set(ngrams(row['text'].split(), 2)))
	else:
		bigrams_appearance_fake.update(set(ngrams(row['text'].split(), 2)))
  
# Añadimos estos datos a los dataframes
bigrams_df['appearance'] = bigrams_df.index.map(bigrams_appearance)
bigrams_real_df['appearance'] = bigrams_real_df.index.map(bigrams_appearance_real)
bigrams_fake_df['appearance'] = bigrams_fake_df.index.map(bigrams_appearance_fake)

In [ ]:
# Añadimos una columna con los bigramas divididos por un espacio
bigrams_df['bigram'] = bigrams_df.index.map(lambda x: ' '.join(x))
bigrams_real_df['bigram'] = bigrams_real_df.index.map(lambda x: ' '.join(x))
bigrams_fake_df['bigram'] = bigrams_fake_df.index.map(lambda x: ' '.join(x))

In [ ]:
# Calculamos el porcentaje de noticias verdaderas que contienen cada bigrama
bigrams_real_df['percentage'] = bigrams_real_df['appearance'] / len(df_clean[df_clean['label']==0]) * 100

In [ ]:
# Hacemos un gráfico de barras de los bigramas más frecuentes en noticias reales
cantidad_bigramas = 25	
plt.figure(figsize=(8, 4))
plt.bar(bigrams_real_df.sort_values(by='percentage', ascending=False).head(cantidad_bigramas)['bigram'], bigrams_real_df.sort_values(by='percentage', ascending=False).head(cantidad_bigramas)['percentage'], color='#6c78d4')
plt.title('Bigramas más aparecidos en noticias reales')
plt.ylim(0, 100)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_yticks(np.arange(0, 110, 10))
plt.xticks(rotation=90)

# Hacemos que en el eje y se muestre el símbolo de porcentaje
vals = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0%}'.format(x/100) for x in vals])

plt.savefig('../images/bigramas_mas_frecuentes_reales_25.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Calculamos el porcentaje de noticias falsas que contienen cada bigrama
bigrams_fake_df['percentage'] = bigrams_fake_df['appearance'] / len(df_clean[df_clean['label']==1]) * 100

# Hacemos un gráfico de barras de los bigramas más frecuentes en noticias falsas
cantidad_bigramas = 25	
plt.figure(figsize=(8, 4))
plt.bar(bigrams_fake_df.sort_values(by='percentage', ascending=False).head(cantidad_bigramas)['bigram'], bigrams_fake_df.sort_values(by='percentage', ascending=False).head(cantidad_bigramas)['percentage'], color='#faed57')
plt.title('Bigramas más aparecidos en noticias falsas')
plt.ylim(0, 100)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_yticks(np.arange(0, 110, 10))
plt.xticks(rotation=90)

# Hacemos que en el eje y se muestre el símbolo de porcentaje
vals = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0%}'.format(x/100) for x in vals])

plt.savefig('../images/bigramas_mas_frecuentes_falsas_25.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Creamos bigram_intersection para guardar los bigramas que aparecen en ambos dataframes
bigrams_intersection = bigrams_real_df.merge(bigrams_fake_df, how='inner', left_index=True, right_index=True, suffixes=('_real', '_fake'))

# Le añadimos el campo 'bigram'
bigrams_intersection['bigram'] = bigrams_intersection.index.map(lambda x: ' '.join(x))

# Calculamos los ratios de aparición de los bigramas en ambos dataframes
bigrams_intersection['ratio_real'] = bigrams_intersection['appearance_real'] / bigrams_intersection['appearance_fake']
bigrams_intersection['ratio_fake'] = bigrams_intersection['appearance_fake'] / bigrams_intersection['appearance_real']

In [ ]:
# Hacemos un gráfico de barras de los bigramas con más ratio de aparición en noticias reales
cantidad_bigramas = 25
plt.figure(figsize=(8, 4))
plt.bar(bigrams_intersection.sort_values(by='ratio_real', ascending=False).head(cantidad_bigramas)['bigram'], bigrams_intersection.sort_values(by='ratio_real', ascending=False).head(cantidad_bigramas)['ratio_real'], color='#6c78d4')
plt.title('Bigramas con mayor ratio de aparición en noticias reales')
plt.ylabel('Ratio')
plt.ylim(0, 500)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_yticks(np.arange(0, 550, 50))
plt.xticks(rotation=90)

plt.savefig('../images/bigramas_mas_ratio_reales_25.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Hacemos un gráfico de barras de los bigramas con más ratio de aparición en noticias falsas
cantidad_bigramas = 25
plt.figure(figsize=(8, 4))
plt.bar(bigrams_intersection.sort_values(by='ratio_fake', ascending=False).head(cantidad_bigramas)['bigram'], bigrams_intersection.sort_values(by='ratio_fake', ascending=False).head(cantidad_bigramas)['ratio_fake'], color='#faed57')
plt.title('Bigramas con mayor ratio de aparición en noticias falsas')
plt.ylabel('Ratio')
plt.ylim(0, 500)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_yticks(np.arange(0, 550, 50))
plt.xticks(rotation=90)

plt.savefig('../images/bigramas_mas_ratio_falsos_25.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Creamos nubes de bigramas más frecuentes en noticias reales y falsas, teniendo en cuenta que los bigramas aún no son strings
# Pasamos los bigramas a strings
bigrams_real_df.index = bigrams_real_df.index.map(lambda x: ' '.join(x))
bigrams_fake_df.index = bigrams_fake_df.index.map(lambda x: ' '.join(x))

# Hacemos un wordcloud de los 20 bigramas más aparecidos en noticias reales que están en bigrams_real_df
wordcloud = WordCloud(width=1920, height=1080, background_color='white', max_words=200).generate_from_frequencies(bigrams_real_df.to_dict()['appearance'], )
plt.figure(figsize=(8,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Bigramas más frecuentes en noticias reales')
plt.savefig('../images/bigramas_mas_frecuentes_reales_wordcloud.png', bbox_inches='tight', dpi=300)
plt.show()

# Hacemos un wordcloud de los 20 bigramas más aparecidos en noticias falsas que están en bigrams_fake_df
wordcloud = WordCloud(width=1920, height=1080, background_color='white', max_words=200).generate_from_frequencies(bigrams_fake_df.to_dict()['appearance'])
plt.figure(figsize=(8,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Bigramas más frecuentes en noticias falsas')
plt.savefig('../images/bigramas_mas_frecuentes_falsas_wordcloud.png', bbox_inches='tight', dpi=300)
plt.show()

#### Trigramas

In [ ]:
# Utilizamos Counter para calcular la frecuencia de los trigramas por label y en global
trigrams = Counter()
trigrams_real = Counter()
trigrams_fake = Counter()
for idx, row in df_clean.iterrows():
	conjunto = set(ngrams(row['text'].split(), 3))
	trigrams.update(conjunto)
	if row['label'] == 0:
		trigrams_real.update(conjunto)
	else:
		trigrams_fake.update(conjunto)

# Pasamos a dataframes con ngramas y cantidad
trigrams_df = pd.DataFrame.from_dict(trigrams, orient='index', columns=['appearance'])
trigrams_real_df = pd.DataFrame.from_dict(trigrams_real, orient='index', columns=['appearance'])
trigrams_fake_df = pd.DataFrame.from_dict(trigrams_fake, orient='index', columns=['appearance'])

In [ ]:
# Borramos algunos objetos para ganar espacio en RAM
del(trigrams)
del(trigrams_real)
del(trigrams_fake)

In [ ]:
# Añadimos una columna con los trigramas divididos por un espacio
trigrams_df['trigram'] = trigrams_df.index.map(lambda x: ' '.join(x))
trigrams_real_df['trigram'] = trigrams_real_df.index.map(lambda x: ' '.join(x))
trigrams_fake_df['trigram'] = trigrams_fake_df.index.map(lambda x: ' '.join(x))

In [ ]:
# Calculamos el porcentaje de noticias verdaderas que contienen cada trigramas
trigrams_real_df['percentage'] = trigrams_real_df['appearance'] / len(df_clean[df_clean['label']==0]) * 100

In [ ]:
# Hacemos un gráfico de barras de los trigramas más frecuentes en noticias reales
cantidad_trigramas = 25
plt.figure(figsize=(8, 4))
plt.bar(trigrams_real_df.sort_values(by='percentage', ascending=False).head(cantidad_trigramas)['trigram'], trigrams_real_df.sort_values(by='percentage', ascending=False).head(cantidad_trigramas)['percentage'], color='#6c78d4')
plt.title('Trigramas más aparecidos en noticias reales')
plt.ylim(0, 100)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_yticks(np.arange(0, 110, 10))
plt.xticks(rotation=90)

# Hacemos que en el eje y se muestre el símbolo de porcentaje
vals = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0%}'.format(x/100) for x in vals])

plt.savefig('../images/trigramas_mas_frecuentes_reales_25.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Calculamos el porcentaje de noticias falsas que contienen cada trigramas
trigrams_fake_df['percentage'] = trigrams_fake_df['appearance'] / len(df_clean[df_clean['label']==1]) * 100

In [ ]:
# Hacemos un gráfico de barras de los trigramas más frecuentes en noticias falsas
cantidad_trigramas = 25
plt.figure(figsize=(8, 4))
plt.bar(trigrams_fake_df.sort_values(by='percentage', ascending=False).head(cantidad_trigramas)['trigram'], trigrams_fake_df.sort_values(by='percentage', ascending=False).head(cantidad_trigramas)['percentage'], color='#faed57')
plt.title('Trigramas más aparecidos en noticias falsas')
plt.ylim(0, 100)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_yticks(np.arange(0, 110, 10))
plt.xticks(rotation=90)

# Hacemos que en el eje y se muestre el símbolo de porcentaje
vals = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0%}'.format(x/100) for x in vals])

plt.savefig('../images/trigramas_mas_frecuentes_falsas_25.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Creamos trigram_intersection para guardar los trigramas que aparecen en ambos dataframes
trigrams_intersection = trigrams_real_df.merge(trigrams_fake_df, how='inner', left_index=True, right_index=True, suffixes=('_real', '_fake'))

# Le añadimos el campo 'trigram'
trigrams_intersection['trigram'] = trigrams_intersection.index.map(lambda x: ' '.join(x))

# Calculamos los ratios de aparición de los trigramas en ambos dataframes
trigrams_intersection['ratio_real'] = trigrams_intersection['appearance_real'] / trigrams_intersection['appearance_fake']
trigrams_intersection['ratio_fake'] = trigrams_intersection['appearance_fake'] / trigrams_intersection['appearance_real']

In [ ]:
# Hacer un gráfico de barras de los trigramas con más ratio de aparición en noticias reales
cantidad_trigramas = 25
plt.figure(figsize=(8, 4))
plt.bar(trigrams_intersection.sort_values(by='ratio_real', ascending=False).head(cantidad_trigramas)['trigram'], trigrams_intersection.sort_values(by='ratio_real', ascending=False).head(cantidad_trigramas)['ratio_real'], color='#6c78d4')
plt.title('Trigramas con mayor ratio de aparición en noticias reales')
plt.ylabel('Ratio')
plt.ylim(0, 250)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_yticks(np.arange(0, 300, 50))
plt.xticks(rotation=90)

plt.savefig('../images/trigramas_mas_ratio_reales_25.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# Hacer un gráfico de barras de los trigramas con más ratio de aparición en noticias falsas
cantidad_trigramas = 25
plt.figure(figsize=(8, 4))
plt.bar(trigrams_intersection.sort_values(by='ratio_fake', ascending=False).head(cantidad_trigramas)['trigram'], trigrams_intersection.sort_values(by='ratio_fake', ascending=False).head(cantidad_trigramas)['ratio_fake'], color='#faed57')
plt.title('Trigramas con mayor ratio de aparición en noticias falsas')
plt.ylabel('Ratio')
plt.ylim(0, 250)
plt.gca().set_axisbelow(True)
plt.gca().yaxis.grid(True, which='major')
plt.gca().set_yticks(np.arange(0, 300, 50))
plt.xticks(rotation=90)

plt.savefig('../images/trigramas_mas_ratio_falsos_25.png', bbox_inches='tight', dpi=300)
plt.show()